In [ ]:
%matplotlib inline


Chatbot Creating
================
**Baseline**: <br>`Matthew Inkawhich <https://github.com/MatthewInkawhich>` <br>
**Implementations and russian adaptation**: <br>
**@h4zzkR** (Telegram)



Preparations
------------



In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

from tqdm import tqdm
import pandas as pd

from torch.utils.data import TensorDataset, DataLoader, Dataset

colab = False
try:
    from google.colab import drive
    drive.mount('/content/drive')
    colab = True
except: 
    pass


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

!pip install pymorphy2[fast]
!pip install -U pymorphy2-dicts-ru

import pymorphy2

Mounted at /content/drive
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 8.2MB 43.6MB/s 
     |████████████████████████████████| 378kB 46.3MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=868306 sha256=28ad7086750532bd125d75ea6c6fe3cb14cc1f96820bb784ff626135c136f823
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG
Requirement already up-to-date: pymorphy2-dicts-ru in /usr/local/lib/python3.7/dist-packages (2.4.417127.4579844)


Load & Preprocess Data
----------------------


In [ ]:
!cp /content/drive/My\ Drive/dl_project/TlkPersonaChatRus.zip /content
#upload from toloka datasets page: 
# https://yandex.ru/blog/toloka/toloka-datasets
!unzip TlkPersonaChatRus.zip

Archive:  TlkPersonaChatRus.zip
   creating: TlkPersonaChatRus/
  inflating: TlkPersonaChatRus/dialogues.tsv  
   creating: __MACOSX/
   creating: __MACOSX/TlkPersonaChatRus/
  inflating: __MACOSX/TlkPersonaChatRus/._dialogues.tsv  
  inflating: TlkPersonaChatRus/readme_TlkPersonaChatRus.txt  
  inflating: __MACOSX/TlkPersonaChatRus/._readme_TlkPersonaChatRus.txt  
  inflating: TlkPersonaChatRus/profiles.tsv  
  inflating: __MACOSX/TlkPersonaChatRus/._profiles.tsv  
  inflating: __MACOSX/._TlkPersonaChatRus  


## Toloka dialoges

In [ ]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

Now we can assemble our vocabulary and query/response sentence pairs.
Before we are ready to use this data, we must perform some
preprocessing.

First, we must convert the Unicode strings to ASCII using
``unicodeToAscii``. Next, we should convert all letters to lowercase and
trim all non-letter characters except for basic punctuation
(``normalizeString``). Finally, to aid in training convergence, we will
filter out sentences with length greater than the ``MAX_LENGTH``
threshold (``filterPairs``).




In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
# filtered_words = [word for word in word_list if word not in stopwords.words('russian')]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def join(chat):
  i = 0
  new_chat = []
  from_ = chat[0][0]
  while True:
    line = [from_, '']
    try:
      while chat[i][0] == from_:
        line[1] += ' ' + chat[i][1] + ' '
        i += 1
    except: 
      new_chat.append(line); break
    from_ = chat[i][0]
    new_chat.append(line)
  return new_chat

def get_user(s):
  return s[s.find('class=')+6:s.find('class=')+19]

In [ ]:
morph = pymorphy2.MorphAnalyzer()

morph_list = [
    'PAD', 'NOUN', 'ADJF', 'ADJS', 'COMP',
    'VERB', 'INFN', 'PRTF', 'PRTS',
    'GRND', 'NUMR', 'ADVB', 'NPRO',
    'PRED', 'PREP', 'CONJ', 'PRCL',
    'INTJ', 'sing', 'plur', 'masc',
    'femn', 'neut', 'LATN', 'PNCT',
    'NUMB', 'intg', 'real', 'UNKN', 'ROMN'
]

morph2int = {m : i for i, m in enumerate(morph_list)}
del morph2int['PAD']

int2morph = {i : m for i, m in enumerate(morph_list)}
del int2morph[0]

MAX_LEN = len(morph2int)

In [ ]:
def morpher(word, leng):
  
  stop = 'VERB INFN GRND NUMR ADVB PRED NPRO PREP COMP CONJ PRCL INTJ'.split()
  data = morph.parse(word)[0]
  leng += 1
  gender, pos, number = 0, 0, 0
  pos = data.tag.POS
  if pos not in stop and pos != None:
    pos = morph2int[pos]
    gender = data.tag.gender
    number = data.tag.number
    try:
      gender = morph2int[gender]
    except: gender = 0
    try:
      number = morph2int[number]
    except: number = 0
  else:
    pos = str(list(data.tag.grammemes)[0])
    try:
      pos = morph2int[pos]
    except: pos = morph2int[data.tag.POS]
  
  return pos, gender, number


def get_morph_list(pair):
    m_pairs = []
    inp, out = pair[0], pair[1]
    m_inp = [morpher(i, MAX_LEN) for i in inp.split()]
    m_out = [morpher(i, MAX_LEN) for i in out.split()]
    print(m_inp, m_out); sys.exit()
    fts = torch.zeros(2, MAX_LEN + 1)
    fts[0][0] = m_inp[0]; fts[0][1] = m_inp[1]; fts[0][2] = m_inp[2]
    fts[1][0] = m_out[0]; fts[1][1] = m_out[1]; fts[1][2] = m_out[2]
      
    return fts

In [ ]:
import sys
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

MAX_LENGTH = 15  # Maximum sentence length to consider
CONTEX_WINDOW = 15

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
  s = re.sub(r"([.!?()_;,:+=-])", r" \1", s.lower())
  s = re.sub('([.,!?()])', r' \1 ', s)
  s = re.sub('\s{2,}', ' ', s)
  s = re.sub(r"[^а-яА-яa-zA-Z.!_;,:+=-?)(,й0123456789]+", r" ", s)
  s = re.sub(r"\s+", r" ", s).strip()
  s = s.replace('br /', ' ').replace('br', ' ')
  s = s.replace('>', ' ').replace('<', ' ').strip()
  return s

def memory_preproc(mem):
  mem = ' '.join(mem).split()
  mem = mem[-CONTEX_WINDOW:len(mem)]
  mem = ' '.join(mem)
  mem = normalizeString(mem)
  mem = tokenizer.tokenize(mem)
  mem = [word for word in mem if word not in stopwords.words('russian')]
  mem = list(dict.fromkeys(mem))
  return ' '.join(mem)
  
  
# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    toloka = pd.read_csv('TlkPersonaChatRus/dialogues.tsv', sep='\t')
    test_pairs = []
    for dlg in tqdm(toloka['dialogue']):
      dlg = dlg.split(sep='</span><br />')
      n_dlg = [[get_user(d), d[d.find(':')+1:].strip()] for d in dlg][:-1]
      n_dlg = [[d[0], normalizeString(d[1])] for d in n_dlg]
      if len(n_dlg) % 2 != 0:
        n_dlg = n_dlg[:-1]
      n_dlg = join(n_dlg)
      for i in range(0, len(n_dlg)-1, 2):
        data = [n_dlg[i][1], n_dlg[i+1][1]]
        data = [normalizeString(d) for d in data]
#         fts = get_morph_list(data)
#         print(fts); sys.exit()
        memory = [d[1] for d in n_dlg[:i+1]]
        memory = memory_preproc(memory)
        test_pairs.append([data, memory])
        
    return test_pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    return len(p[0][0].split(' ')) < MAX_LENGTH and len(p[0][1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    a = [pair for pair in pairs if filterPair(pair)]
    return a
  
  
def loadPrepareData(corpus, corpus_name, voc):
    print("Start preparing training data ...")
    test_pairs = readVocs(corpus, corpus_name)
    print("Read {!s} sentence pairs".format(len(test_pairs)))
    test_pairs = filterPairs(test_pairs)
    
    print("Trimmed to {!s} sentence pairs".format(len(test_pairs)))
    print("Counting words...")
    for pair in test_pairs:
        voc.addSentence(pair[0][0])
        voc.addSentence(pair[0][1])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, test_pairs


corpus_name = 'xl_vocab'
voc = Voc(corpus_name)

corpus = 'TlkPersonaChatRus/dialogues.tsv'
voc, toloka = loadPrepareData(corpus, 'toloka', voc)
print("\npairs:")
for pair in  toloka[:10]:
    print(pair[0])

Start preparing training data ...
Reading lines...


  2%|▏         | 233/10013 [00:02<01:31, 106.32it/s]


KeyboardInterrupt: ignored

## Github dataset

In [ ]:
# def read_text(replies):
#   replies = [normalizeString(r) for r in replies]
#   if len(replies) % 2 != 0:
#     replies = replies[:-1]
#   pairs = []
#   for i in range(0, len(replies)-1, 2):
#     data = [replies[i], replies[i+1]]
#     memory = [d for d in replies[:i+1]]
#     memory = memory_preproc(memory)
#     pairs.append([data, memory])
#   return pairs

In [ ]:
# with open('dialogues.txt') as file:
#   f = file.read().split('\n\n')
#   dataset = []
#   for dialog in tqdm(f):
#     replies = dialog.split('\n')
#     replies = read_text(replies)
#     dataset.append(replies)

# def filter_Pair(p):
#     try:
#       return len(p[0][0][0].split(' ')) < MAX_LENGTH and len(p[0][0][1].split(' ')) < MAX_LENGTH
#     except IndexError:
#       return False

# # Filter pairs using filterPair condition
# def filter_Pairs(pairs):
#     a = [pair for pair in pairs if filter_Pair(pair)]
#     return a

# github = []
# for d in dataset:
#   try:
#     github.append(d[0])
#   except IndexError: continue
    
# github = filter_Pairs(github)
# print("Trimmed to {!s} sentence pairs".format(len(github)))
# print("Counting words...")

# for pair in github:
#   voc.addSentence(' '.join(pair[0][0]))
#   voc.addSentence(' '.join(pair[0][1]))
#   voc.addSentence(' '.join(pair[1]))
    
# print("Counted words:", voc.num_words)

**Trim rare words**

In [ ]:
# combined = toloka + github
combined = toloka

In [ ]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, morph_pairs, MIN_COUNT):
    voc.trim(MIN_COUNT)
    keep_pairs = []
    for i in range(len(pairs)):
        pair = pairs[i]
        input_sentence = pair[0][0]
        output_sentence = pair[0][1]
        memory = pair[1]
        keep_input = True
        keep_output = True
        keep_memory = True

        for word in input_sentence.split(' '):
            if word not in voc.word2index or word == 'готовить':
                keep_input = False
                break
        for word in output_sentence.split(' '):
            if word not in voc.word2index or word == 'готовить':
                keep_output = False
                break
                
        for word in memory.split(' '):
          if word not in voc.word2index or word == 'готовить':
              keep_memory = False
              break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output and keep_memory:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


dataset = trimRareWords(voc, combined, morph_pairs, MIN_COUNT)

Trimmed from 49622 pairs to 34811, 0.7015 of total


In [ ]:
dataset[0]

[['привет ) расскажи о себе',
  'привет ) под вкусный кофеек настроение поболтать появилось )'],
 'привет расскажи',
 ([tensor([ 1., 21., 18.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.]),
   tensor([24.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.]),
   tensor([28.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.]),
   tensor([14.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.]),
   tensor([14.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  

Prepare Data for Models
-----------------------


In [ ]:
MAX_LENGTH = max([len(i[0][0].split()) for i in dataset])
MAX_LENGTH = max([MAX_LENGTH] + [len(i[0][1].split()) for i in dataset])
MAX_LENGTH = MAX_LENGTH + 1

In [ ]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths
  
def memoryVar(mem, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in mem]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0][0].split(" ")), reverse=True)
    input_batch, memory_batch, output_batch = [], [], []
    for pair in pair_batch:
        input_batch.append(pair[0][0])
        output_batch.append(pair[0][1])
        memory_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    mem, mem_lengths = memoryVar(memory_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, mem, mem_lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(dataset) for _ in range(small_batch_size)])
input_variable, lengths, mem, mem_lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print('memory: ', mem)
print('memory lens: ', mem_lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  103,  4501,  3476,   569,     3],
        [ 4315,    25,    45,  5777,     2],
        [ 1114,    17,   103,     2,     0],
        [   25,    21,  1029,     0,     0],
        [   56,     4,  1083,     0,     0],
        [   57,     4,     2,     0,     0],
        [ 1379,     2,     0,     0,     0],
        [15292,     0,     0,     0,     0],
        [ 3352,     0,     0,     0,     0],
        [   29,     0,     0,     0,     0],
        [ 6566,     0,     0,     0,     0],
        [   78,     0,     0,     0,     0],
        [ 1027,     0,     0,     0,     0],
        [    4,     0,     0,     0,     0],
        [    2,     0,     0,     0,     0]])
lengths: tensor([15,  7,  6,  3,  2])
memory:  tensor([[  103,   282,   113,  1643,     3],
        [ 4315,   102,   139,  1706,     2],
        [ 1114,   283,   140,   509,     0],
        [ 1379,  3716,   830,   248,     0],
        [15292,  4501,  3476,  3231,     0],
        [ 3352,     2,   103,   569,

# Model

**Pipeline**
--------------------
**Input_data** is reply ('Hello') and 'memory'.<br>
**Memory** is fixed length previous user and bot replyes without stopwords.<br>

-----------------------
Special encoder (general encoder = gencoder) is responsible for detecting dialog context, with help of it<br>
seq2seq model is trying to create message. Hidden state from gencoder is using for encoder rnn.<br>
In the decoder section, we count attention memory weights and attention context weights (gencoder_outputs and encoder_outputs). <br>
After that, we concat rnn_output, context, memory_context (hiddensize*3) and push it to linear.


In [ ]:
class PositionalEmbedding(nn.Module):
  """
  embedding that consider 
  words position in sentence
  """
  def __init__(self, emb_dim, max_seq_len):
      super().__init__()
      self.emb_dim = emb_dim

      pe = torch.zeros(max_seq_len, emb_dim)
      for pos in range(max_seq_len):
          for i in range(0, emb_dim, 2):
              pe[pos, i] = \
              math.sin(pos / (10000 ** ((2 * i)/emb_dim)))
              pe[pos, i + 1] = \
              math.cos(pos / (10000 ** ((2 * (i + 1))/emb_dim)))

      pe = pe.unsqueeze(0)
      self.register_buffer('pe', pe)
        
  def forward(self, x):
      x = x * math.sqrt(self.emb_dim)
      #add constant to embedding
      seq_len = x.shape[0]
      x = x.permute(1, 0, 2)
      x = x + Variable(self.pe[:,:seq_len],
                       requires_grad=False)
      return x
      
class Embedding(nn.Module):
  def __init__(self, vocab_len, emb_dim, max_seq_len=100):
    super(Embedding, self).__init__()
    self.emb_dim = emb_dim
    self.v_len = vocab_len
    self.embedding = nn.Embedding(self.v_len, self.emb_dim, padding_idx=0)
    self.pos_encode = PositionalEmbedding(emb_dim, max_seq_len)
    
  def forward(self, x):
    b = self.embedding(x)
    return self.pos_encode(b)

In [ ]:
class GlobalEncoder(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(GlobalEncoder, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)
        


    def forward(self, input_seq, input_lengths, hidden=None):
        embedded = self.embedding(input_seq)
        embedded = embedded.permute(1, 0, 2)

        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths, enforce_sorted=False)
        outputs, hidden = self.gru(packed, hidden)

        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        return outputs, hidden

In [ ]:
class Encoder(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, memory_out=0, memory_hidden=None):

        embedded = self.embedding(input_seq)
        embedded = embedded.permute(1, 0, 2)

        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(packed, memory_hidden)
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)

        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        return outputs, hidden

In [ ]:
# Luong attention layer
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method

        self.hidden_size = hidden_size
        self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        
    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def forward(self, hidden, encoder_outputs, mask): #+mask
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        attn_energies = attn_energies.t()
        #masked attention
        attn_energies[~mask] = float('-inf')

        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(Decoder, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        
        self.dense_bn = nn.BatchNorm1d(hidden_size)
        self.concat = nn.Linear(hidden_size * 3, hidden_size)
        
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs, memory_out, memory_hidden,
                                                                memory_mask, encoder_mask):

        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        embedded = embedded.permute(1, 0, 2)

        rnn_output, hidden = self.gru(embedded, last_hidden)

        attn_weights = self.attn(rnn_output, encoder_outputs, encoder_mask)
        memory_weights = self.attn(rnn_output, memory_out, memory_mask)

        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        memory_context = memory_weights.bmm(memory_out.transpose(0, 1))

        rnn_output = rnn_output.squeeze(0)
        
        context = context.squeeze(1)
        memory_context = memory_context.squeeze(1)

        concat_input = torch.cat((rnn_output, context, memory_context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        concat_output = self.dense_bn(concat_output)

        output = self.out(concat_output)
        output = F.softmax(output, dim=1)

        return output, hidden

Define Training Procedure
-------------------------

Masked loss
~~~~~~~~~~~

Since we are dealing with batches of padded sequences, we cannot simply
consider all elements of the tensor when calculating loss. We define
``maskNLLLoss`` to calculate our loss based on our decoder’s output
tensor, the target tensor, and a binary mask tensor describing the
padding of the target tensor. This loss function calculates the average
negative log likelihood of the elements that correspond to a *1* in the
mask tensor.




In [ ]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [ ]:
def train(input_variable, lengths, memory, memory_lens, target_variable, mask, max_target_len, gencoder, encoder, decoder, embedding,
          gencoder_optimizer, encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    gencoder_optimizer.zero_grad()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    memory = memory.to(device)
    memory_lens = memory_lens.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    loss = 0
    print_losses = []
    n_totals = 0

    memory_mask = memory > 0
    memory_mask = memory_mask.permute(1, 0).to(device)
    encoder_mask = input_variable > 0
    encoder_mask = encoder_mask.permute(1, 0).to(device)

    gencoder_outputs, gencoder_hidden = gencoder(memory, memory_lens)
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths,  gencoder_outputs, gencoder_hidden)

    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    decoder_hidden = encoder_hidden[:decoder.n_layers]

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs,
                gencoder_outputs, gencoder_hidden, memory_mask,
                encoder_mask
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = torch.nn.utils.clip_grad_norm_(gencoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    gencoder_optimizer.step()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [ ]:
def trainIters(model_name, voc, pairs, gencoder, encoder, decoder, gencoder_optimizer, encoder_optimizer, decoder_optimizer, 
               embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, 
               print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, memory, memory_lens, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, memory, memory_lens, target_variable, mask, max_target_len, gencoder, encoder,
                     decoder, embedding, gencoder_optimizer, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            torch.save({
                'iteration': iteration,
                'gen' : gencoder.state_dict(),
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'gen_opt' : gencoder_optimizer.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join('/content/', '{}_{}.tar'.format(iteration, 'checkpoint')))

Run Model
---------

Finally, it is time to run our model!

Regardless of whether we want to train or test the chatbot model, we
must initialize the individual encoder and decoder models. In the
following block, we set our desired configurations, choose to start from
scratch or set a checkpoint to load from, and build and initialize the
models. Feel free to play with different model configurations to
optimize performance.




In [ ]:
# !cp /content/drive/My\ Drive/10_checkpoint.tar /content/
!cp 1_checkpoint.tar /content/drive/My\ Drive/

In [ ]:
# Configure models
model_name = 'cb_model'
# attn_model = 'dot'
attn_model = 'general'
#attn_model = 'concat'

#small hiddens can`t be effecient in loss and predict
#for such a big vocabulary

hidden_size = 512
encoder_n_layers = 3
decoder_n_layers = 2
dropout = 0.2
batch_size = 32

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 500

# loadFilename = os.path.join('/content/2000_checkpoint.tar')


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    gencoder_sd = checkpoint['gen']
    gencoder_optimizer_sd = checkpoint['gen_opt']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = Embedding(voc.num_words, hidden_size)

if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models

gencoder = GlobalEncoder(hidden_size, embedding, encoder_n_layers, dropout)
encoder = Encoder(hidden_size, embedding, encoder_n_layers, dropout)
decoder = Decoder(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)

if loadFilename:
    gencoder.load_state_dict(gencoder_sd)
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device

gencoder = gencoder.to(device)
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

In [ ]:
start, end = 1500, 4500
for i in range(start, end + save_every, save_every):
  d = str(i) + '_checkpoint.tar'
  !rm $d

In [ ]:
from torch.autograd import Variable

clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0

#try 10000+, bcz of size of dataset
n_iteration = 1000
print_every = 100
save_every = 1000

save_dir = '/content/'
corpus_name = 'corp'

# Ensure dropout layers are in train mode
gencoder.train()
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
gencoder_optimizer = optim.Adam(gencoder.parameters(), lr=learning_rate * 2.5)
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, dataset, gencoder, encoder, decoder, gencoder_optimizer, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 100; Percent complete: 10.0%; Average loss: 1.7875
Iteration: 200; Percent complete: 20.0%; Average loss: 1.8306
Iteration: 300; Percent complete: 30.0%; Average loss: 1.8440
Iteration: 400; Percent complete: 40.0%; Average loss: 1.8327
Iteration: 500; Percent complete: 50.0%; Average loss: 1.7820
Iteration: 600; Percent complete: 60.0%; Average loss: 1.7731
Iteration: 700; Percent complete: 70.0%; Average loss: 1.7774
Iteration: 800; Percent complete: 80.0%; Average loss: 1.7179
Iteration: 900; Percent complete: 90.0%; Average loss: 1.7566
Iteration: 1000; Percent complete: 100.0%; Average loss: 1.7080


# Chatbot

Define Evaluation
-----------------


In [ ]:
def evaluate(gencoder, encoder, decoder, searcher, voc, sentence, memory, max_length=MAX_LENGTH):
  
    memory_lengths = 0
    has_memory = False
    
    memory = [indexesFromSentence(voc, ' '.join(memory))]
    memory_lengths = torch.tensor([len(indexes) for indexes in memory])
    memory = torch.LongTensor(memory).transpose(0, 1)
    memory.to(device)
    memory_lengths.to(device)
    has_memory = True
      
    indexes_batch = [indexesFromSentence(voc, sentence)]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)

    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    tokens, scores = searcher(input_batch, lengths, max_length, memory, memory_lengths, has_memory)
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words
  

class MemoryMechanism():
  def __init__(self):
    self.memory = []
    self.limit = CONTEX_WINDOW
      
  def append_input(self, input_seq):
    if len(self.memory) < self.limit-1:
      data = input_seq
      memory = [normalizeString(d) for d in data]
      memory = memory_preproc(memory)
      memory = [token for token in memory if token in voc.word2index.keys()]
      self.memory += memory
    else:
      self.memory.pop(0)
    
  def drop(self):
    self.memory = []

def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    memory_class = MemoryMechanism()
    while(1):
        try:
            input_sentence = input('> ')
            if input_sentence == 'q' or input_sentence == 'quit': break
            if input_sentence == 'clear': memory_class.drop(); continue
            input_sentence = normalizeString(input_sentence)
            memory_class.append_input(input_sentence.split())
            memory = memory_class.memory
            output_words = evaluate(gencoder, encoder, decoder, searcher, voc, input_sentence, memory)
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))
            memory_class.append_input(output_words)

        except KeyError:
            print("Error: Encountered unknown word.")

In [ ]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, gencoder, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.gencoder = gencoder
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length, memory_seq, memory_lengths, has_memory=True):
      
        memory_seq, memory_lengths = memory_seq.to(device), memory_lengths.to(device)
        if has_memory:
          gencoder_outputs, gencoder_hidden = self.gencoder(memory_seq, memory_lengths)
        else:
          gencoder_outputs, gencoder_hidden = None, None
        
        gencoder_outputs, gencoder_hidden = gencoder_outputs.to(device), gencoder_hidden.to(device)
          
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length, gencoder_outputs, gencoder_hidden)
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        
        memory_mask = memory_seq > 0
        memory_mask = memory_mask.permute(1,0).to(device)
        
        encoder_mask = input_seq > 0
        encoder_mask = encoder_mask.permute(1, 0).to(device)

        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        for _ in range(max_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, 
                                                          encoder_outputs, gencoder_outputs, gencoder_hidden,
                                                          memory_mask, encoder_mask)
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        return all_tokens, all_scores

In [ ]:
# Set dropout layers to eval mode
gencoder.eval()
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(gencoder, encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, voc)

> приветик
Bot: привет )
> что делаешь?
Bot: люблю , а вы ?
> я тоже)
Bot: ооо , это круто


KeyboardInterrupt: ignored

# Exxtra

**Telegram chat bot**: <br>
[Here you can find *it*](https://t.me/stupid_chat_arc4num_bot)

In [ ]:
!pip install python-telegram-bot --upgrade

In [ ]:
def chat_input(input_seq, gencoder, encoder, decoder, searcher, voc):
    input_sentence = ''
    memory_class = MemoryMechanism()
    try:
        input_sentence = input_seq
        input_sentence = normalizeString(input_sentence)
        memory_class.append_input(input_sentence.split())
        memory = memory_class.memory
        output_words = evaluate(gencoder, encoder, decoder, searcher, voc, input_sentence, memory)
        output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
        memory_class.append_input(output_words)
        response = ' '.join(output_words)
        return response

    except KeyError:
        print("Error: Encountered unknown word.")

In [ ]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
updater = Updater(token='877251641:AAGF32236-QIlbIjufWHXkBAEwG8DOX9M3U')
dispatcher = updater.dispatcher

# Обработка команд
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Привет, я чат-бот.')
    
def textMessage(bot, update):
    message = update.message.text
    response = chat_input(message, gencoder, encoder, decoder, searcher, voc)
    bot.send_message(chat_id=update.message.chat_id, text=response)



start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

In [ ]:
!mv 1000_checkpoint.tar 6000_iters_good.tar
!cp 6000_iters_good.tar /content/drive/My\ Drive/